In [1]:
import requests, json
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
# get data from FPL API
FPL_Data=requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()

# show the fields
pprint(FPL_Data, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 7590201}


In [3]:
# get players info from 'elements' field
players = pd.json_normalize(FPL_Data['elements'])

players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,80201,-1,1,-1,1,0,1,1.0,...,429,44,205,13,NaN,,NaN,,NaN,
1,75.0,0.0,115918,0,0,0,0,0,1,-0.4,...,258,10,329,30,NaN,,NaN,,NaN,
2,50.0,50.0,47431,0,0,-1,1,0,3,-0.2,...,553,230,556,231,4.0,,2.0,,NaN,
3,100.0,50.0,54694,0,0,-1,1,0,4,0.0,...,114,27,240,32,NaN,,NaN,,1.0,
4,NaN,NaN,58822,0,0,0,0,0,2,0.0,...,319,126,197,68,NaN,,5.0,,NaN,


In [4]:
# get teams info from 'teams' field
teams = pd.json_normalize(FPL_Data['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1250,1130,1150,1220,1210,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1100,1160,1120,1140,1080,1100,2
2,94,0,None,3,0,Brentford,0,0,0,BRE,...,None,False,0,1010,1020,1020,1030,1020,1030,130
3,36,0,None,4,0,Brighton,0,0,0,BHA,...,None,False,0,1100,1130,1150,1190,1100,1130,131
4,90,0,None,5,0,Burnley,0,0,0,BUR,...,None,False,0,1060,1070,1040,1090,1040,1080,43


In [5]:
# add teams to players
complete_fpl = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)
# show result
complete_fpl[['first_name', 'second_name', 'name']]

,first_name,second_name,name
0,Bernd,Leno,Arsenal
1,Rúnar Alex,Rúnarsson,Arsenal
2,Willian,Borges Da Silva,Arsenal
3,Pierre-Emerick,Aubameyang,Arsenal
4,Cédric,Soares,Arsenal
...,...,...,...
562,José,Malheiro de Sá,Wolves
563,Patrick,Cutrone,Wolves
564,Luke,Cundle,Wolves
565,Chem,Campbell,Wolves


In [6]:
# get position info from 'element_types' field
positions = pd.json_normalize(FPL_Data['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],60
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],195
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],236
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],76


In [7]:
# add position to players
complete_fpl = complete_fpl.merge(
    positions,
    left_on='element_type',
    right_on='id'
)
# rename columns
complete_fpl=complete_fpl.rename(
columns={'name':'team_name', 'singular_name':'position_name'})

# show result 
complete_fpl[
    ['first_name', 'second_name', 'team_name', 'position_name']
]

,first_name,second_name,team_name,position_name
0,Bernd,Leno,Arsenal,Goalkeeper
1,Rúnar Alex,Rúnarsson,Arsenal,Goalkeeper
2,Karl,Hein,Arsenal,Goalkeeper
3,Aaron,Ramsdale,Arsenal,Goalkeeper
4,Jed,Steer,Aston Villa,Goalkeeper
...,...,...,...,...
562,Max,Kilman,Wolves,Defender
563,Ki-Jana,Hoever,Wolves,Defender
564,Yerson,Mosquera Valdelamar,Wolves,Defender
565,Rayan,Ait Nouri,Wolves,Defender


In [8]:
# begin request data for previous years performance by requesting API
FPL_Data= requests.get('https://fantasy.premierleague.com/api/element-summary/1/').json()

# show API fields
pprint(FPL_Data, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [9]:
def get_player_history(player_id):
    '''get historical season info for a given player_id'''
    
    # request general info from FPL API
    FPL_Data = requests.get(
            'https://fantasy.premierleague.com/api/element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from dataframe
    complete_fpl = pd.json_normalize(FPL_Data['history_past'])
    
    return complete_fpl


# show player #1's season history
get_player_history(1).head()

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,2018/19,80201,50,49,106,2835,0,0,6,42,...,0,0,0,105,5,568,807.2,0.0,0.0,80.5
1,2019/20,80201,50,48,114,2649,0,0,7,39,...,0,2,0,113,10,591,843.2,0.0,0.0,84.1
2,2020/21,80201,50,50,131,3131,0,0,11,37,...,0,0,1,86,11,625,702.2,0.0,2.0,70.3


In [10]:
# select necessary columns from players dataframe
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type', 'code']
]

# join team name to players
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join positions to players
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

In [11]:
# get season histories for each player
points = players['id_player'].apply(get_player_history)

points = pd.concat(complete_fpl for complete_fpl in points)

points.head()

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,2018/19,80201,50,49,106,2835,0,0,6,42,...,0,0,0,105,5,568,807.2,0.0,0.0,80.5
1,2019/20,80201,50,48,114,2649,0,0,7,39,...,0,2,0,113,10,591,843.2,0.0,0.0,84.1
2,2020/21,80201,50,50,131,3131,0,0,11,37,...,0,0,1,86,11,625,702.2,0.0,2.0,70.3
0,2020/21,115918,45,44,1,15,0,0,0,0,...,0,0,0,2,0,8,16.6,0.0,0.0,1.7
0,2016/17,225321,40,40,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [12]:
# rename code on players df to align with points df
players=players.rename(
columns={'code':'element_code', 'singular_name_short': 'position'})
players.head()

,id_player,first_name,second_name,web_name,element_code,name,position
0,1,Bernd,Leno,Leno,80201,Arsenal,GKP
1,2,Rúnar Alex,Rúnarsson,Rúnarsson,115918,Arsenal,GKP
2,532,Karl,Hein,Hein,463748,Arsenal,GKP
3,559,Aaron,Ramsdale,Ramsdale,225321,Arsenal,GKP
4,28,Jed,Steer,Steer,79852,Aston Villa,GKP


In [13]:
# remove unwanted columns from players tab as I only want to add the 'web_name' column to points tab
players = players[
    ['web_name', 'element_code', 'position']
]
players.head()

,web_name,element_code,position
0,Leno,80201,GKP
1,Rúnarsson,115918,GKP
2,Hein,463748,GKP
3,Ramsdale,225321,GKP
4,Steer,79852,GKP
